In [68]:
import random
import string
from typing import List
from sqlalchemy import  create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker

from app.data_generator import data_gen_sqla

In [15]:
DATABASE_CONFIG = {
    "host": "localhost",
    "port": "3306",
    "user": "instaqp",
    "password": "passw0rd",
    "database": "instaqp",
    "dialect": "mysql",
    "connector": "mysqlconnector"
}


In [16]:
Base = declarative_base()

In [17]:
# es test model


class Elasticsearch(Base):

    __tablename__ = "elasticsearch"
    __table_args__ = {"extend_existing": True}

    id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
    address = Column(String(255), nullable=False)
    age = Column(Integer, nullable=True)

    def __repr__(self) -> str:
        return self.name


In [40]:
SQLALCHEMY_DATABASE_URI = "{}+{}://{}:{}@{}:{}/{}".format(
    DATABASE_CONFIG["dialect"],
    DATABASE_CONFIG["connector"],
    DATABASE_CONFIG["user"],
    DATABASE_CONFIG["password"],
    DATABASE_CONFIG["host"],
    DATABASE_CONFIG["port"],
    DATABASE_CONFIG["database"],
)
engine = create_engine(SQLALCHEMY_DATABASE_URI)


In [41]:
# CREATE THE TABLES
Base.metadata.create_all(engine)

In [52]:
Session = sessionmaker(bind=engine)
session = Session()

In [53]:
def data_gen_sqla(limit: int) -> List:
    temp_data_store = []
    for i in range(limit):
        document = {
            "name": "".join(random.choices(string.ascii_letters, k=15)),
            "address": "".join(random.choices(string.ascii_letters, k=15)),
            "age": random.randint(a=10, b=99),
        }
        temp_data_store.append(document)
    return temp_data_store


In [55]:
data = data_gen_sqla(limit=10)
for i in data:
    print(i)
    es_obj = Elasticsearch(name=i["name"], address=i["address"], age=i["age"])
    session.add_all([es_obj])
    session.commit()

{'name': 'bJtEqEbezmvnPmU', 'address': 'ASyAGCjjzZXKXmN', 'age': 39}
{'name': 'mBOAswhgZiTavev', 'address': 'TvepmJBSBRlSMjH', 'age': 83}
{'name': 'bsKgHEEdLhGfyFR', 'address': 'gdIBEVRMTmYKKjg', 'age': 89}
{'name': 'cIvcOzdDBmUhDPR', 'address': 'yPzLRYTAQtYdaKz', 'age': 19}
{'name': 'iVSwrSXFTJzEwbN', 'address': 'gZojGlIPBdVZKkl', 'age': 96}
{'name': 'JtLDAarYhLmdpBk', 'address': 'sxSGRAcLFVKSCmw', 'age': 88}
{'name': 'OQETspFHIsmMcvR', 'address': 'bVhPqinUDlpNeHG', 'age': 27}
{'name': 'yiXnZqwGIGYPLGU', 'address': 'NSHVEjmnkmvlxeW', 'age': 96}
{'name': 'zcLJWlssZUEQqWC', 'address': 'dZhleTguBxRiWaz', 'age': 92}
{'name': 'TFuIrjUOOvTNJUj', 'address': 'gYvSiKcjaBaoypZ', 'age': 63}


In [67]:
import json


query = session.query(Elasticsearch)
values = query.all()
for i in values:
    print(i.age)

68
39
83
89
19
96
88
27
96
92
63


In [71]:
inspector = inspect(engine)
inspector.get_schema_names()
inspector.get_table_names()

['elasticsearch', 'elasticsearch_test', 'tags']